Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-10 17:27:05--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-10 17:27:06--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K  58.9KB/s    in 1.1s    

2021-10-10 17:27:08 (58.9 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [ ]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/uc?id=1wZh0BLJckv3J7k3KLIHW0uxZU2TI_565

Downloading...
From: https://drive.google.com/uc?id=1wZh0BLJckv3J7k3KLIHW0uxZU2TI_565
To: /content/2018EE10957_A_model.zip
100% 17.6M/17.6M [00:00<00:00, 47.9MB/s]


In [ ]:
!unzip 2018EE10957_A_model.zip

Archive:  2018EE10957_A_model.zip
   creating: data/
  inflating: data/best-biLSTM-params.pt  
  inflating: data/Field_TEXT         
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.csv    
  inflating: data/train/train.data.txt  
  inflating: data/biLSTM_train.log   
   creating: data/validation/
  inflating: data/validation/validation.data.txt  
  inflating: data/validation/validation.gold.txt  
  inflating: data/validation/validation.csv  


In [1]:
import os
import pandas
import numpy as np
import dill
import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchtext.legacy import data

In [2]:
## Various utility functions

class args_class:
    dataset = "data/"
    model = "biLSTM"
    gpu = 0
    batch_size = 32
    epochs = 20
    hidden_dim = 128
    lr = 0.001

In [3]:
def parse_args():
	# parser = ArgumentParser(description='NLI Baseline')
	# parser.add_argument('--dataset', '-d', type=str, default='mnli')
	# parser.add_argument('--model', '-m', type=str, default='bilstm')
	# parser.add_argument('--gpu', type=int, default=0)
	# parser.add_argument('--batch_size', type=int, default=128)
	# parser.add_argument('--embed_dim', type=int, default=300)
	# parser.add_argument('--d_hidden', type=int, default=200)
	# parser.add_argument('--dp_ratio', type=int, default=0.2)
	# parser.add_argument('--epochs', type=int, default=20)
	# parser.add_argument('--lr', type=float, default=0.001)
	# parser.add_argument('--combine', type=str, default='cat')
	# parser.add_argument('--results_dir', type=str, default='results')
	args = args_class()
	return check_args(args)

"""checking arguments"""
def check_args(args):
	# --result_dir
	check_folder(os.path.join(args.dataset))

	# --epoch
	try:
			assert args.epochs >= 1
	except:
			print('number of epochs must be larger than or equal to one')

	# --batch_size
	try:
			assert args.batch_size >= 1
	except:
			print('batch size must be larger than or equal to one')
	return args

def get_device(gpu_no):
	if torch.cuda.is_available():
		torch.cuda.set_device(gpu_no)
		return torch.device('cuda:{}'.format(gpu_no))
	else:
		return torch.device('cpu')

def makedirs(name):
	"""helper function for python 2 and 3 to call os.makedirs()
		avoiding an error if the directory to be created already exists"""

	import os, errno

	try:
		os.makedirs(name)
	except OSError as ex:
		if ex.errno == errno.EEXIST and os.path.isdir(name):
			# ignore existing directory
			pass
		else:
			# a different error happened
			raise

def check_folder(log_dir):
	if not os.path.exists(log_dir):
		os.makedirs(log_dir)
	return log_dir

def get_logger(args, phase):
	logging.basicConfig(level=logging.INFO, 
												filename = "{}/{}_{}.log".format(args.dataset, args.model, phase),
												format = '%(asctime)s - %(message)s', 
												datefmt='%d-%b-%y %H:%M:%S')
	return logging.getLogger(phase)

In [4]:
import spacy
en = spacy.load('en_core_web_sm')
def tokeni(sen):
    t = en.tokenizer(sen)
    return [word.text for word in t]

In [5]:
class myModel(nn.Module):
    def __init__(self, pretrained_embeddings, embed_dim=100, hidden_dim=100, device=torch.device('cpu')):
        super(myModel, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.device = device
        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=False)
        self.dropout = nn.Dropout(p = 0.5)
        self.bilstm = nn.LSTM(embed_dim, hidden_dim, bidirectional=False, batch_first=True, num_layers=1)
        self.similarity = nn.CosineSimilarity(dim=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        '''
            [torchtext.legacy.data.batch.Batch of size 32]
                [.word]:[torch.LongTensor of size 1x32]
                [.POS]:[torch.LongTensor of size 32]
                [.sen1]:[torch.LongTensor of size 17x32]
                [.word1]:[torch.LongTensor of size 32]
                [.sen2]:[torch.LongTensor of size 26x32]
                [.word2]:[torch.LongTensor of size 32]
                [.label]:[torch.LongTensor of size 32]
        '''
        input1 = input.sen1.permute(1,0).to(self.device)
        input2 = input.sen2.permute(1,0).to(self.device)

        # embed the tokens (indices in the vocab) 
        out1 = self.dropout(self.embedding(input1))
        out2 = self.dropout(self.embedding(input2))

        out1, (_, _) = self.bilstm(out1)
        out2, (_, _) = self.bilstm(out2)

        # word of interest
        woi1_idx = torch.repeat_interleave(torch.unsqueeze(torch.unsqueeze(input.word1, 1), 2), self.hidden_dim, dim=2).to(self.device)
        woi2_idx = torch.repeat_interleave(torch.unsqueeze(torch.unsqueeze(input.word2, 1), 2), self.hidden_dim, dim=2).to(self.device)

        # gather the word of interest for each sentence in the batch
        out1 = torch.gather(input=out1, dim=1, index=woi1_idx).squeeze()
        out2 = torch.gather(input=out2, dim=1, index=woi2_idx).squeeze()
        
        # compute scores of similarity
        out = self.similarity(out1, out2)
        out = self.sigmoid(out)
        # try:
        #     assert out.shape[0] == self.bs
        # except:
        #     print("AssertionError", out.shape[0], self.bs) # all samples are touched in a epoch

        return out


In [ ]:
def main():
    hidden_dim=128
    device = get_device(0)

    x_test = pandas.read_csv("test.data.txt", delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    word1 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    x_test = x_test.drop("position", axis=1)
    x_test.loc[:, "POS"] = x_test.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    x_test.insert(3, "word1", word1)
    x_test.insert(5, "word2", word2)
    x_test.to_csv("data/test.csv", header=None, index=False)

    with open(os.path.join("data", "Field_TEXT"), 'rb') as f:
        TEXT = dill.load(f)

    fields = [
        ('word', TEXT),
        ('POS', data.Field(use_vocab=False, sequential=False)), 
        ('sen1', TEXT),
        ('word1', data.Field(use_vocab=False, sequential=False)),
        ('sen2', TEXT),
        ('word2', data.Field(use_vocab=False, sequential=False)),
    ]

    test_set = data.TabularDataset.splits(
        path = "data",
        test = "test.csv",
        format = 'csv',
        fields = fields,
        skip_header = False
    )

    (test_itr,) = data.BucketIterator.splits(
        test_set,
        #sort_key = lambda sample : len(sample.sen1),
        sort = False,
        batch_size = 32,
        repeat = False
    )

    model = myModel(pretrained_embeddings=TEXT.vocab.vectors, embed_dim=TEXT.vocab.vectors.shape[1], hidden_dim=hidden_dim, device=device)
    saved_model = torch.load("data/best-biLSTM-params.pt", map_location=device)
    print(saved_model["accuracy"])
    model.load_state_dict(saved_model['model_dict'])
    model = model.to(device)

    model.eval()
    test_itr.init_epoch()
    predictions = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(test_itr):
            answer = model(batch)
            answer = list(np.array((answer>=0.5).long().cpu()))
            predictions += answer

    predictions = pandas.DataFrame(predictions)[0].apply(lambda label : "T" if label == 1 else "F")
    predictions.to_csv("output.txt", header=None, index=None)
    print("predictions written in output.txt")

In [ ]:
main()

56.739811912225704
predictions written in output.txt


In [ ]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-10 17:36:14--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-10 17:36:15--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt.2’

test.gold.txt.2     100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-10 17:36:16 (81.8 MB/s) - ‘test.gold.txt.2’ saved [1276/1276]

Performance =  0.542319749216301
